<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_LangGraph_Exercise/blob/main/RAG_Chatbot/try_langgraph_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%%capture --no-stderr
%pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters playwright unstructured
!playwright install

In [11]:
from google.colab import userdata

# load the environment variables set in colab
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

## Retriever

In [26]:
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

import asyncio

# use Air Canada sample accessibility URLs as examples
urls = [
    "https://www.westjet.com/en-ie/special-needs/assistance-services",
]

loader = SeleniumURLLoader(urls=urls)
docs = loader.load()
# docs_list = [item for sublist in docs for item in sublist]

# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=100, chunk_overlap=50
# )
# doc_splits = text_splitter.split_documents(docs_list)

# # Add to vectorDB
# vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=OpenAIEmbeddings(),
# )
# retriever = vectorstore.as_retriever()

In [27]:
for doc in docs:
    print(doc)
    break

page_content='Skip to content

The browser you are using is not supported. Please see our supported browsers.

We are currently experiencing high call volumes. Please only call if you are travelling within the next 72 hours.

Please visit our Self-serve page for available options

By using the WestJet website, you agree to the website terms of use, the privacy statement, and the use of cookies.

Assistance services

Many assistance services are available on WestJet operated flights. The details of services are outlined in each section, and upon request, we are pleased to:

check you in at the counter.

allow you to move to the front of the line at a check-in if you are unable to use an automated a self-service kiosk.

provide you with, if you are in a wheelchair or are not independently mobile, a place to wait that is near our staff while you wait to board, or if you are transferring to another flight. Check with you periodically to inquire about your needs and attend to those needs (a